<a href="https://colab.research.google.com/github/none-304/Coursera_Capstone/blob/main/Finance_Methods_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import packages
import numpy as np

In [2]:
import pandas as pd

In [3]:
from google.colab import files

In [4]:
from google.colab import drive

In [5]:
#Preparing Google Drive
drive.mount('/content/drive')

##### If you are using the attached file, upload the file using the code below #### 
# import files
# from google.colab import files
# files.upload()

##### If you are using python natively, just change the path in the cell below to the path of the file ####

Mounted at /content/drive


In [6]:
#Load up dataset
a=pd.read_sas('/content/drive/MyDrive/w23handout.sas7bdat')


In [7]:
#Checking the columns
a.columns

Index(['GVKEY', 'DATADATE', 'CONM', 'SIC', 'SICH', 'LPERMNO', 'AT', 'SALE'], dtype='object')

In [8]:
### 1st task


#Specifying Assets and Sale parameter
a=a[(a.AT>=5)& (a.SALE>0)]

In [9]:
#importing progress bar
import tqdm

In [10]:
#Prepare blank dataframe
c=pd.DataFrame()

In [11]:
a['SALE']

0           25.395
1           32.007
2           53.798
3           13.793
4           13.829
            ...   
272613    2085.000
272614       0.934
272615     177.987
272617       8.203
272619     512.094
Name: SALE, Length: 233663, dtype: float64

In [12]:
a['SALE'].shift(+1)

0              NaN
1           25.395
2           32.007
3           53.798
4           13.793
            ...   
272613    2541.000
272614    2085.000
272615       0.934
272617     177.987
272619       8.203
Name: SALE, Length: 233663, dtype: float64

In [13]:
#Detaching each data based on firm code to create lagged variable & appending it back into one
for b in tqdm.tqdm(set(a.GVKEY)):
  z=a[a.GVKEY==b]
  z['LAGSALE']=z['SALE'].shift(+1)
  z['GSALE']=(z.SALE-z.LAGSALE)/z.LAGSALE
  z['LAGASSETS']=z['AT'].shift(+1)
  z['GASSETS']=(z.AT-z.LAGASSETS)/z.LAGASSETS
  c=c.append(z)



  0%|          | 0/22517 [00:00<?, ?it/s]<ipython-input-13-ffbf2e97b60b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z['LAGSALE']=z['SALE'].shift(+1)
<ipython-input-13-ffbf2e97b60b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z['GSALE']=(z.SALE-z.LAGSALE)/z.LAGSALE
<ipython-input-13-ffbf2e97b60b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [14]:
#Retain firms that have 3 or more data points
c=c.groupby('GVKEY').filter(lambda x : len(x)>=3)

In [15]:
#Specifying datadate
c=c[c.DATADATE.dt.year.between(1985,2021)]

In [16]:
c

,GVKEY,DATADATE,CONM,SIC,SICH,LPERMNO,AT,SALE,LAGSALE,GSALE,LAGASSETS,GASSETS
160103,b'023226',2005-12-31,b'INUVO INC',b'7370',7311.0,90608.0,57.494,40.441,NaN,NaN,NaN,NaN
160104,b'023226',2006-12-31,b'INUVO INC',b'7370',7311.0,90608.0,120.397,71.882,40.441,0.777454,57.494,1.094079
160105,b'023226',2007-12-31,b'INUVO INC',b'7370',7311.0,90608.0,121.126,73.577,71.882,0.023580,120.397,0.006055
160106,b'023226',2008-12-31,b'INUVO INC',b'7370',7311.0,90608.0,29.220,51.573,73.577,-0.299061,121.126,-0.758764
160107,b'023226',2009-12-31,b'INUVO INC',b'7370',7311.0,90608.0,25.825,51.655,51.573,0.001590,29.220,-0.116188
...,...,...,...,...,...,...,...,...,...,...,...,...
200409,b'032315',2017-12-31,b'TDH HOLDINGS INC',b'2040',2040.0,16931.0,19.811,28.980,NaN,NaN,NaN,NaN
200410,b'032315',2018-12-31,b'TDH HOLDINGS INC',b'2040',2040.0,16931.0,17.429,23.674,28.980,-0.183092,19.811,-0.120236
200411,b'032315',2019-12-31,b'TDH HOLDINGS INC',b'2040',2040.0,16931.0,15.087,12.648,23.674,-0.465743,17.429,-0.134374
200412,b'032315',2020-12-31,b'TDH HOLDINGS INC',b'2040',2040.0,16931.0,18.453,0.815,12.648,-0.935563,15.087,0.223106


In [17]:
#Dropping data points that have null values in the growth variables
c.replace([np.inf,np.nan],inplace=True)
c.dropna(subset=['GSALE','GASSETS'],inplace=True)

In [18]:
##### 2nd task


#Loading up SIC to fama french key database
sic_ff12=pd.read_csv('/content/drive/MyDrive/SIC_to_Fama_French_industry.csv')

In [19]:
#Merging the dataset into one
finaldf=pd.merge(c,sic_ff12,left_on=c.SIC.astype(int),right_on='SIC0')
finaldf

,GVKEY,DATADATE,CONM,SIC_x,SICH,LPERMNO,AT,SALE,LAGSALE,GSALE,LAGASSETS,GASSETS,SIC_y,SIC0,FF_12,FF_48
0,b'023226',2006-12-31,b'INUVO INC',b'7370',7311.0,90608.0,120.397,71.882,40.441,0.777454,57.494,1.094079,7370,7370,6,34
1,b'023226',2007-12-31,b'INUVO INC',b'7370',7311.0,90608.0,121.126,73.577,71.882,0.023580,120.397,0.006055,7370,7370,6,34
2,b'023226',2008-12-31,b'INUVO INC',b'7370',7311.0,90608.0,29.220,51.573,73.577,-0.299061,121.126,-0.758764,7370,7370,6,34
3,b'023226',2009-12-31,b'INUVO INC',b'7370',7311.0,90608.0,25.825,51.655,51.573,0.001590,29.220,-0.116188,7370,7370,6,34
4,b'023226',2010-12-31,b'INUVO INC',b'7370',7311.0,90608.0,14.026,48.970,51.655,-0.051979,25.825,-0.456883,7370,7370,6,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198611,b'025921',2004-12-31,b'RAILAMERICA INC',b'4013',4013.0,78065.0,1016.143,395.564,358.381,0.103753,1232.490,-0.175537,4013,4013,12,40
198612,b'025921',2005-12-31,b'RAILAMERICA INC',b'4013',4013.0,78065.0,1147.376,423.682,395.564,0.071083,1016.143,0.129148,4013,4013,12,40
198613,b'025921',2009-12-31,b'RAILAMERICA INC',b'4013',4013.0,93068.0,1598.629,410.603,423.682,-0.030870,1147.376,0.393291,4013,4013,12,40
198614,b'025921',2010-12-31,b'RAILAMERICA INC',b'4013',4013.0,93068.0,1603.653,490.291,410.603,0.194076,1598.629,0.003143,4013,4013,12,40


In [20]:
#removing the fama french category 8, 11, & 12
finaldf=finaldf[(finaldf.FF_12!=8)& (finaldf.FF_12!=11) & (finaldf.FF_12!=12)]

In [21]:
finaldf

,GVKEY,DATADATE,CONM,SIC_x,SICH,LPERMNO,AT,SALE,LAGSALE,GSALE,LAGASSETS,GASSETS,SIC_y,SIC0,FF_12,FF_48
0,b'023226',2006-12-31,b'INUVO INC',b'7370',7311.0,90608.0,120.397,71.882,40.441,0.777454,57.494,1.094079,7370,7370,6,34
1,b'023226',2007-12-31,b'INUVO INC',b'7370',7311.0,90608.0,121.126,73.577,71.882,0.023580,120.397,0.006055,7370,7370,6,34
2,b'023226',2008-12-31,b'INUVO INC',b'7370',7311.0,90608.0,29.220,51.573,73.577,-0.299061,121.126,-0.758764,7370,7370,6,34
3,b'023226',2009-12-31,b'INUVO INC',b'7370',7311.0,90608.0,25.825,51.655,51.573,0.001590,29.220,-0.116188,7370,7370,6,34
4,b'023226',2010-12-31,b'INUVO INC',b'7370',7311.0,90608.0,14.026,48.970,51.655,-0.051979,25.825,-0.456883,7370,7370,6,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198594,b'211531',2002-12-31,b'ATLAS PEARLS LTD',b'0900',900.0,83923.0,13.340,6.174,5.693,0.084490,10.830,0.231764,900,900,1,48
198595,b'211531',2003-12-31,b'ATLAS PEARLS LTD',b'0900',900.0,83923.0,16.688,6.706,6.174,0.086168,13.340,0.250975,900,900,1,48
198596,b'211531',2004-12-31,b'ATLAS PEARLS LTD',b'0900',900.0,83923.0,16.536,4.545,6.706,-0.322249,16.688,-0.009108,900,900,1,48
198597,b'211531',2005-12-31,b'ATLAS PEARLS LTD',b'0900',900.0,83923.0,17.995,7.084,4.545,0.558636,16.536,0.088232,900,900,1,48


In [22]:
#grouping by year
finaldf.groupby(finaldf.DATADATE.dt.year).agg(['mean','count']).loc[:,['GASSETS','GSALE']]

GASSETS           GSALE      
              mean count      mean count
DATADATE                                
1985      0.137352  3020  0.258011  3020
1986      0.165856  3074  0.379779  3074
1987      0.178418  3184  0.461324  3184
1988      0.131211  3124  0.484138  3124
1989      0.137466  3075  0.420037  3075
1990      0.111826  3068  0.381380  3068
1991      0.085746  3201  0.215213  3201
1992      0.119314  3466  1.245720  3466
1993      0.150223  3840  0.387905  3840
1994      0.169856  4108  1.889741  4108
1995      0.215048  4316  0.614140  4316
1996      0.254391  4723  0.486824  4723
1997      0.214101  4821  0.794048  4821
1998      0.214126  4694  0.516266  4694
1999      0.254703  4543  0.465020  4543
2000      0.380228  4458  0.810567  4458
2001      0.072063  4138  0.603995  4138
2002      0.022031  3871  0.139368  3871
2003      0.105738  3627  0.357794  3627
2004      0.199403  3577  0.555645  3577
2005      0.141226  3491  0.675619  3491
2006      0.187236  3414  0.270575  3414
2007      0.179829  3291  2.111715  3291
2008      0.047512  3110  0.546678  3110
2009      0.059159  2958  0.250080  2958
2010      0.147636  2886  3.492238  2886
2011      0.167980  2818  0.417974  2818
2012      0.126919  2768  0.416416  2768
2013      0.122587  2796  0.254792  2796
2014      0.145379  2834  1.216348  2834
2015      0.100619  2805  0.921695  2805
2016      0.083161  2751  3.382879  2751
2017      0.130398  2709  0.976453  2709
2018      0.139103  2715  5.344632  2715
2019      0.149666  2705  0.665314  2705
2020      0.283623  2590  2.419750  2590
2021      0.299550  2472  2.670717  2472